- Batch currently not working, as repo is private!

<a target="_blank" href="https://colab.research.google.com/github/mlde-ms/vision-transformer-from-scratch.git">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

from vit import VisionTransformer
from mnist_training import train
from mnist_evaluation import evaluate
from visualize_attention import visualize_attention_head

train = False

In [ ]:
model = VisionTransformer(28, 2, 2, 2, 64, 256, num_classes=10, representation_size=32)
if (train):
    train(model)

# Expects model weights to be stored in `vit_weights.pth`
evaluate(model)

In [ ]:
# model = VisionTransformer(28, 2, 2, 2, 64, 256, num_classes=10, representation_size=32)
# model.load_state_dict(torch.load('vit_weights.pth'), map_location=torch.device('cpu'))

model.eval()
model = model.cpu()

# Define the transform to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

batch_size = 10

# Download and load the MNIST test data
test_data = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
# Create a data loader for the test data
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Get a batch of test data and iterate over it
images, labels = next(iter(test_loader))
for i in range(batch_size):

    image = images[i]
    label = labels[i].item()

    # Add a dummy batch dimension of 1
    image = image[None]
    # Get the model prediction
    out = model(image)

    for layer in range(2):
        for head in range(2):
            attention_output = model.get_attention_output(image, layer, head)
            visualize_attention_head(attention_output[0], 196, 2, f'layer_{layer}_head_{head}.png', f'attention/{i}_{label}')

    original_image = images[i].squeeze().numpy()
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')
    plt.savefig(f'attention/{i}_{label}/img.png', bbox_inches='tight')
    plt.close()